## Run QIIME2 with BBSRC Data: Batch/Plate 3

Ref: https://gist.github.com/tkosciol/29de5198a4be81559a075756c2490fde

Using Jupyter Lab Notebook and QIIME2 Python API.

In [96]:
import pandas as pd

More imports than we really need...

In [97]:
import qiime2
from tempfile import mkdtemp
from qiime2.plugins import demux, deblur, quality_filter, \
                           metadata, feature_table, alignment, \
                           phylogeny, diversity, emperor, feature_classifier, \
                           taxa, composition

In [98]:
from qiime2.plugins.dada2.methods import denoise_paired

In [99]:
from matplotlib import pyplot as plt

In [100]:
%matplotlib inline

In [101]:
import os, tarfile, wget

Set default figure size for matplotlib

In [102]:
plt.rcParams['figure.figsize'] = [9.5, 6]

## Get data

Use *trimmed* data

Local copy of data files are in folder '/mnt/j/4BBSRC/data_plate3/Trimmed/'

Had to rename files and remove duplicate (low read count) samples.

In [103]:
pth = '/mnt/j/4BBSRC/data_plate3/Trimmed/'

#must match '.+_.+_L[0-9][0-9][0-9]_R[12]_001\\.fastq\\.gz'

In [105]:
!ls /mnt/j/4BBSRC/data_plate3/Trimmed/ | head

1-455-061F_220506_L001_R1_001.fastq.gz
1-455-061F_220506_L001_R2_001.fastq.gz
10-455-064F_220506_L001_R1_001.fastq.gz
10-455-064F_220506_L001_R2_001.fastq.gz
11-455-064A_220506_L001_R1_001.fastq.gz
11-455-064A_220506_L001_R2_001.fastq.gz
12-455-064B_220506_L001_R1_001.fastq.gz
12-455-064B_220506_L001_R2_001.fastq.gz
13-455-065F_220506_L001_R1_001.fastq.gz
13-455-065F_220506_L001_R2_001.fastq.gz


In [17]:
#CasavaOneEightSingleLanePerSampleDirFmt fails??
mydata = qiime2.Artifact.import_data('SampleData[PairedEndSequencesWithQuality]',pth, view_type='CasavaOneEightSingleLanePerSampleDirFmt')
mydata.save('QB3Data')
fn = 'QB3Data.qza'

## Get Metadata

Study metadata

In [93]:
#!mamba install openpyxl -c conda-forge -y

In [140]:
#mdf=pd.read_excel('clinical_data_Dec22.xlsx')
#mdf.set_index('ID',inplace=True)
#mdf

Build full metadata from file list

In [156]:
mdf = pd.DataFrame(os.listdir(pth),columns=['pth'])

mdf['key']=mdf['pth'].str.extract('^([0-9]{1,2}-.*)_[0-9]{6}.*')
mdf['idx']=mdf['key'].str.extract('(^[0-9]{1,2}).*')
#mdf['subject']=mdf['pth'].str.extract('^[0-9]{1,2}-(.*)_[0-9]{6}.*')
mdf['site']=mdf['key'].str.extract('^[0-9]{1,2}-[0-9]{3}-[0-9]{3}([A-Z])$')
mdf['subject']=mdf['key'].str.extract('^[0-9]{1,2}-(.*)[A-Z]{0,1}$')
mdf['subject']=mdf['subject'].str.replace('-','_')
mdf.set_index('key',drop=True,inplace=True)
mdf.drop('pth',axis=1,inplace=True)
mdf.drop_duplicates(inplace=True)
mdf.index.name='id'
mdf

,idx,site,subject
id,,,
1-455-061F,1,F,455_061F
10-455-064F,10,F,455_064F
11-455-064A,11,A,455_064A
12-455-064B,12,B,455_064B
13-455-065F,13,F,455_065F
...,...,...,...
92-PCRPos,92,NaN,PCRPos
93-455-049B,93,B,455_049B
94-455-013A,94,A,455_013A


In [157]:
mdf[mdf.index.str.contains('PCR')]

,idx,site,subject
id,,,
91-PCRNeg,91,NaN,PCRNeg
92-PCRPos,92,NaN,PCRPos


In [159]:
metad = qiime2.Metadata(mdf)

In [160]:
smdat = demux.visualizers.summarize(mydata)
#smdat.visualization.save('qb3datv')
smdatv = smdat.visualization

/home/mcdssso/miniconda3/envs/qiime2-2022.11/lib/python3.8/site-packages/q2_demux/_summarize/_visualizer.py:191: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  context['result_data'] = context['result_data'].append(df)
/home/mcdssso/miniconda3/envs/qiime2-2022.11/lib/python3.8/site-packages/q2_demux/_summarize/_visualizer.py:191: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  context['result_data'] = context['result_data'].append(df)


<Figure size 950x600 with 0 Axes>

In [161]:
smdatv

<visualization: Visualization uuid: 5995e0b1-18e5-4f18-9e8a-cdda0878d45e>

In [162]:
# !jupyter serverextension enable --py qiime2 --sys-prefix
# then restart server...

Are the parameters correct here?

In [ ]:
qc = denoise_paired(mydata,220,220,20,20)
qc_table = qc.table
qc_denoising_stats = qc.denoising_stats
qc_representative_sequences = qc.representative_sequences
qc_table.save('qb3qc_table')
#qc_denoising_stats.save('qb3qc_denoising_stats')
#qc_representative_sequences.save('qb3qc_representative_sequences')
del qc

Running external command line application(s). This may print messages to stdout and/or stderr.
The command(s) being run are below. These commands cannot be manually re-run as they will depend on temporary files that no longer exist.

Command: run_dada.R --input_directory /tmp/tmpss4_igfu/forward --input_directory_reverse /tmp/tmpss4_igfu/reverse --output_path /tmp/tmpss4_igfu/output.tsv.biom --output_track /tmp/tmpss4_igfu/track.tsv --filtered_directory /tmp/tmpss4_igfu/filt_f --filtered_directory_reverse /tmp/tmpss4_igfu/filt_r --truncation_length 220 --truncation_length_reverse 220 --trim_left 20 --trim_left_reverse 20 --max_expected_errors 2.0 --max_expected_errors_reverse 2.0 --truncation_quality_score 2 --min_overlap 12 --pooling_method independent --chimera_method consensus --min_parental_fold 1.0 --allow_one_off False --num_threads 1 --learn_min_reads 1000000

R version 4.2.2 (2022-10-31) 


Loading required package: Rcpp


DADA2: 1.26.0 / Rcpp: 1.0.9 / RcppParallel: 5.1.6 
2) Filtering 

In [71]:
df = qc_table.view(pd.DataFrame)

In [72]:
df.head()

,472099d0ca288082f6e509d8486840ac,c4ffc9cda21e2d895072c17a5589f0f4,f0309e23b34e31994c8ecdeff35fcf8c,116050b6ca29bbed7ae0197e5e2b4ad0,fd3f596d5b8d142ac7bd6dcb1cfb1c0a,8c77ffb75eef05f9fe0cbab537a35f5b,fa74111ef8ea771c10dd7d9e398a7742,8243edde812e09c18ed7932c4cdcac93,bd38d16291f2c9dc6ea0c020eaa3fb8d,a73fdbab84d9cd75ade0dd68daab46f0,...,b719b620ecb371a53520fb8b3af3209a,ea8132b3cc398e37e2fd4ad644d86cd3,5f5701cf26c6ff1414dbdd9dbb17714b,4e667751f9d21df50dbd08416028fba7,fbc4416feb3ef62740e5ae48c1b4cd97,0e7cabf55e09d9eba0ca3a1c825f027a,bedfb96fe23c239f5f896db4371f32dc,a496419c42782b99bd91a66832d15467,10c99dc71685cde894b9b9b8b2fe8cb8,6ef915bf6507e7caba8bf8b8bcd2a880
1-455-061F,23828.0,0.0,7.0,175.0,0.0,789.0,0.0,0.0,0.0,138.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10-455-064F,33030.0,0.0,291.0,741.0,0.0,742.0,0.0,122.0,0.0,1053.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11-455-064A,5867.0,0.0,427.0,1350.0,0.0,742.0,0.0,244.0,0.0,258.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12-455-064B,13697.0,0.0,1486.0,2707.0,0.0,639.0,15.0,530.0,0.0,1177.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13-455-065F,24400.0,0.0,67.0,703.0,0.0,463.0,0.0,0.0,0.0,884.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [73]:
statsmeta = metadata.visualizers.tabulate(input=qc_denoising_stats.view(qiime2.Metadata))

In [74]:
statsmeta.visualization

<visualization: Visualization uuid: 57a76350-178c-4a70-8d19-197eaacb17b3>

In [75]:
qcv = metadata.visualizers.tabulate(qc_table.view(qiime2.Metadata))

In [76]:
qcv.visualization

<visualization: Visualization uuid: 1963cb47-9304-4401-a9d3-ae959eecf564>

In [77]:
ftv = feature_table.visualizers.summarize(qc_table)

In [78]:
ftv.visualization

<visualization: Visualization uuid: 6acc7bd4-e810-42f7-b4de-bcf4d43006fe>

### From Paul

See: https://docs.qiime2.org/2022.2/tutorials/feature-classifier/

In [79]:
import sklearn
sklearn.__version__

'0.24.1'

In [80]:
repref = qiime2.Artifact.load('97_otus.qza')

In [81]:
taxref = qiime2.Artifact.load('ref-taxonomy.qza')

In [82]:
cnb = feature_classifier.methods.fit_classifier_naive_bayes(repref,taxref)
#cnb.classifier.save('qb3class')
cnbclass = cnb.classifier


/home/mcdssso/miniconda3/envs/qiime2-2022.11/lib/python3.8/site-packages/q2_feature_classifier/classifier.py:102: UserWarning: The TaxonomicClassifier artifact that results from this method was trained using scikit-learn version 0.24.1. It cannot be used with other versions of scikit-learn. (While the classifier may complete successfully, the results will be unreliable.)
  warnings.warn(warning, UserWarning)


In [83]:
tax = feature_classifier.actions.classify_sklearn(qc_representative_sequences,cnbclass)
tax.classification.save('qb3taxclass')
taxclass = tax.classification
del tax

In [84]:
taxv = metadata.visualizers.tabulate(taxclass.view(qiime2.Metadata))

In [85]:
taxv.visualization

<visualization: Visualization uuid: e64c76fa-1296-4210-bf75-b12c1c2d6dd9>

In [86]:
taxa_bar_plot = qiime2.plugins.taxa.visualizers.barplot(qc_table, taxclass, metad)
taxa_bar_plot.visualization

<visualization: Visualization uuid: c8b645a6-b38b-4a9e-b89c-a5e00afd438f>

Do a pure Python plot

---